# Pre-training Tiny-GPT-2 on BabyLM data

**This notebook is based on Tutorials provided by Optimum Graphcore**

Pre-training GPT-2 on BabyLM requires:

* BabyLM data that can be loaded via:  
```
git clone https://github.com/upunaprosk/small-language-models.git
cd small-language-models
bash download_data.sh
```

* This notebooks assumes you have trained a tokenizer on the corpus you are using, see the [How to train a tokenizer](https://github.com/huggingface/notebooks/blob/master/examples/tokenizer_training.ipynb) notebook.

First of all, make sure your environment has installed the latest version of [🤗 Optimum Graphcore](https://github.com/huggingface/optimum-graphcore).


In order to improve usability and support for future users, Graphcore would like to collect information about the
applications and code being run in this notebook. The following information will be anonymised before being sent to Graphcore:

- User progression through the notebook
- Notebook details: number of cells, code being run and the output of the cells
- Environment details

You can disable logging at any time by running `%unload_ext gc_logger` from any cell.

In [4]:
%pip install -q "optimum-graphcore==0.6.1"
%pip install examples-utils[common]@git+https://github.com/graphcore/examples-utils@latest_stable
from examples_utils import notebook_logging
%load_ext gc_logger

Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/graphcore/examples-utils (to revision latest_stable) to /tmp/pip-install-zqkpf2l7/examples-utils_d6ebb1f115d14b40a97c6fb86c9555cc
  Running command git clone --filter=blob:none --quiet https://github.com/graphcore/examples-utils /tmp/pip-install-zqkpf2l7/examples-utils_d6ebb1f115d14b40a97c6fb86c9555cc
  Running command git checkout -q 40c62e6646db8f9d60d1707a61204c95a15c7ccb
  Resolved https://github.com/graphcore/examples-utils to commit 40c62e6646db8f9d60d1707a61204c95a15c7ccb
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.
Loading extensions from ~/.ipython/extensions is deprecated. We recommend managing extensions like any other Python packages, in site-packages.
In order to improve usability and support for future users, Graphcore would like to collect information about the applications and code being run in this notebook. The fo

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [8]:
# from huggingface_hub import notebook_login

# notebook_login()

Then you need to install Git-LFS:

In [9]:
# !apt install git-lfs

Let's print out the versions of Transformers and Optimum Graphcore:

In [10]:
import transformers
import optimum.graphcore
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML


print(transformers.__version__)
print(optimum.graphcore.__version__)

4.25.1
0.6.1


Values for machine size and cache directories can be configured through environment variables or directly in the notebook:

In [11]:
import os

pod_type = os.getenv("GRAPHCORE_POD_TYPE", "pod4")
executable_cache_dir = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "/tmp/exe_cache/") + "/language_modelling_from_scratch"

# Train a language model

In this notebook, we'll see how to train a [🤗 Transformers](https://github.com/huggingface/transformers) model on a language modeling task. We will cover two types of language modeling tasks which are:

- Causal language modeling: the model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it to access the tokens after token i when trying to predict the token i+1 in the sentence.

![Widget inference representing the causal language modeling task](images/causal_language_modeling.png)

- Masked language modeling: the model has to predict some tokens that are masked in the input. It still has access to the whole sentence, so it can use the tokens before and after the tokens masked to predict their value.

![Widget inference representing the masked language modeling task](images/masked_language_modeling.png)

We will see how to easily load and preprocess the dataset for each one of those tasks, and how to use the `IPUTrainer` API to train a model on it.

## Preparing the dataset

For each of those tasks, we will use the [Wikitext 2]() dataset as an example. You can load it very easily with the 🤗 Datasets library.

In [12]:
from datasets import load_dataset
from pathlib import Path
files=[p.as_posix() for p in Path('./babylm_data/babylm_10M/').glob('*.train')]
files_dev=[p.as_posix() for p in Path('./babylm_data/babylm_dev/').glob('*.dev')]
print(files_dev)
d = load_dataset('text', data_files={'train': list(files), 'eval': list(files_dev)})

Found cached dataset text (/tmp/huggingface_caches/datasets/text/default-d8fd0c9a344ad3d7/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


['babylm_data/babylm_dev/open_subtitles.dev', 'babylm_data/babylm_dev/simple_wikipedia.dev', 'babylm_data/babylm_dev/cbt.dev', 'babylm_data/babylm_dev/gutenberg.dev', 'babylm_data/babylm_dev/switchboard.dev', 'babylm_data/babylm_dev/aochildes.dev', 'babylm_data/babylm_dev/qed.dev', 'babylm_data/babylm_dev/wikipedia.dev', 'babylm_data/babylm_dev/children_stories.dev', 'babylm_data/babylm_dev/bnc_spoken.dev']


  0%|          | 0/2 [00:00<?, ?it/s]

You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. Just uncomment the following cell and replace the paths with values that will lead to your files:

In [15]:
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [16]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [17]:
show_random_elements(d["train"])

,text
0,"Please, please!"
1,"For the three points, let's see if you are correct."
2,Emacs versus vi?
3,Er
4,"Good evening, everyone!"
5,Yeah.
6,"Not for drinking, but to trap a gangster by name Tiger Bhai."
7,That's what that sound is.
8,go over to him and show him .
9,That was my grandfather's name.


As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

## Causal Language modeling

For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text that may look like:
```
part of text 1
```
or 
```
end of text 1 [BOS_TOKEN] beginning of text 2
```
depending on whether they span over several of the original texts in the dataset or not. The labels will be the same as the inputs, shifted to the left.

We will use the [`gpt2`](https://huggingface.co/gpt2) architecture for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=causal-lm) as long as that model is supported by Optimum Graphcore. The IPU config files of the supported models are available in Graphcore's [Hugging Face account](https://huggingface.co/Graphcore). You can also create your own IPU config file locally. For the tokenizer, you can replace the checkpoint by the one you trained yourself.

In this notebook, we are using both data parallelism and pipeline parallelism (see this [tutorial](https://github.com/graphcore/tutorials/tree/master/tutorials/pytorch/efficient_data_loading) for more). Therefore the global batch size, which is the actual number of samples used for the weight update, is determined with three factors:
- global batch size = micro_batch_size * gradient accumulation steps * replication factor

and replication factor is determined by `pod_type`, which will be used as a key to select the replication factor from a dictionary defined in the IPU config file. For example, the dictionary in the IPU config file [Graphcore/gpt2-small-ipu](https://huggingface.co/Graphcore/gpt2-small-ipu/blob/main/ipu_config.json) looks like this:
- "replication_factor": {"pod4": 1, "pod8": 2, "pod16": 4, "pod32": 8, "pod64": 16, "default": 1}

Depending on you model and the pod machine you are using, you might need to adjust these three batch-size-related arguments.

In [18]:
from transformers import GPT2TokenizerFast
path_tokenizer_config="custom_tokenizer_gpt_2.json"
tokenizer = GPT2TokenizerFast(vocab_file=None,
                                     merges_file=None,
                                     tokenizer_file=path_tokenizer_config,
                                     )

In [19]:
model_checkpoint = "gpt2"
ipu_config_name = "Graphcore/gpt2-small-ipu"
micro_batch_size = 1
gradient_accumulation_steps = 64
dataloader_workers = 64

To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [21]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized_datasets = d.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
print(f'Length of train data={len(tokenized_datasets["train"])}')

Loading cached processed dataset at /tmp/huggingface_caches/datasets/text/default-d8fd0c9a344ad3d7/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2/cache-81de5b7289ea3c29_*_of_00004.arrow
Loading cached processed dataset at /tmp/huggingface_caches/datasets/text/default-d8fd0c9a344ad3d7/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2/cache-227037c1b68f519b_*_of_00004.arrow


Length of train data=1058740


Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

In [22]:
tokenized_datasets["train"][1],tokenized_datasets["eval"][1]

({'input_ids': [2391,
   11146,
   2109,
   29820,
   6648,
   18456,
   325,
   1287,
   16655,
   1412,
   36,
   20090,
   7751,
   3095,
   27251,
   34,
   4407,
   3075,
   37,
   354,
   167],
  'attention_mask': [1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1]},
 {'input_ids': [7201, 12, 391, 471, 444, 524, 31],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1]})

Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

First, we grab the maximum length our model was pretrained with. This might be too big to fit on your IPU RAM, so here we take a bit less at just 128.

In [23]:
# block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our texts:

In [24]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.

Also note that by default, the `map` method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of `block_size` every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower). You can also speed-up the preprocessing by using multiprocessing:

In [25]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=64,
    num_proc=4,
)

Loading cached processed dataset at /tmp/huggingface_caches/datasets/text/default-d8fd0c9a344ad3d7/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2/cache-0825af2c0192db43_*_of_00004.arrow
Loading cached processed dataset at /tmp/huggingface_caches/datasets/text/default-d8fd0c9a344ad3d7/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2/cache-ad033bdbebe5df8b_*_of_00004.arrow


And we can check our datasets have changed: now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original texts.

In [26]:
tokenizer

PreTrainedTokenizerFast(name_or_path='', vocab_size=30000, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'})

In [27]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

"TH A PRlCE. JUST ASK ClNDERELLA, WHO GAVE UP HER FlRSTBORN TO BECOME A PRlNCESS. DlD YOU GET EVERYTHlNG  YOU DESlRED? BUT NO ONE KNOWS MORE ABOUT THE PRlCE OF MAGlC THAN THE DARK WlZARD HlMSELF-- RUMPLESTlLTSKlN. ALL MAGlC COMES WlTH A PRlCE. (Granny) IT'S ALL HERE. WHY, YES, OF COURSE IT"

To instantiate an `IPUTrainer`, we will need to define three more things. The first thing we need to define is the `IPUConfig`, which is a class that specifies attributes and configuration parameters to compile and put the model on the device. We initialize it with one config name or path, which we set earlier. We also get the model configuration from the model name set earlier and initialize our model using that config.

In [28]:
from transformers import GPT2Config
model_checkpoint
GPT2Config()

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.25.1",
  "use_cache": true,
  "vocab_size": 50257
}

In [29]:
from transformers import AutoConfig, AutoModelForCausalLM
from optimum.graphcore import IPUConfig, IPUTrainer, IPUTrainingArguments

# ipu_config = IPUConfig.from_pretrained(ipu_config_name, executable_cache_dir=executable_cache_dir)

ipu_config = IPUConfig.from_dict({
  "_mode": "training",
  "auto_loss_scaling": False,
  "device_iterations": 2,
  "embedding_serialization_factor": 4,
  "enable_half_partials": True,
  "executable_cache_dir": "/tmp/exe_cache/3.2.1/language_modelling_from_scratch",
  "execute_encoder_on_cpu_for_generation": False,
  "gradient_accumulation_steps": 16,
  "inference_device_iterations": 5,
  "inference_embedding_serialization_factor": 1,
  "inference_ipus_per_replica": 4,
  "inference_layers_per_ipu": [
    2,
    2,
    2,
    2
  ],
  "inference_matmul_proportion": 0.25,
  "inference_projection_serialization_factor": 1,
  "inference_replication_factor": 1,
  "inference_serialized_embedding_splits_per_ipu": None,
  "inference_serialized_projection_splits_per_ipu": None,
  "ipus_per_replica": 4,
  "layers_per_ipu": [
    0,
    2,
    2,
    2
  ],
  "matmul_proportion": 0.25,
  "optimizer_state_offchip": True,
  "optimum_version": "1.6.1",
  "output_mode": "final",
  "projection_serialization_factor": 1,
  "recompute_checkpoint_every_layer": True,
  "replicated_tensor_sharding": True,
  "replication_factor": 1,
  "seed": None,
  "serialized_embedding_splits_per_ipu": None,
  "serialized_projection_splits_per_ipu": None,
  "transformers_version": "4.25.1"
})

# ipu_config = IPUConfig.from_pretrained(
#     ipu_config_name,
#     executable_cache_dir=executable_cache_dir,
#     ipus_per_replica=2,
#     layers_per_ipu= [
#     0,
#     2,
#     2,
#     2
#   ],
#     inference_layers_per_ipu=[-1]
# )

`replicated_tensor_sharding` is not used when `replication_factor=1`


In [30]:
OPT_CONFIG={
  "activation_function": "gelu_new",
  "attn_pdrop": 0.2,
  "embd_pdrop": 0.2,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 768,
  "n_head": 12,
  "n_inner": None,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": False,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": False,
  "scale_attn_weights": True,
  "summary_activation": None,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": True,
  "summary_type": "cls_index",
  "summary_use_proj": True,
  "transformers_version": "4.25.1",
  "use_cache": True,
  "vocab_size": tokenizer.vocab_size
}
config = GPT2Config(**OPT_CONFIG)
config.update({'activation_function':'gelu'})
config


GPT2Config {
  "activation_function": "gelu",
  "attn_pdrop": 0.2,
  "bos_token_id": 50256,
  "embd_pdrop": 0.2,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.25.1",
  "use_cache": true,
  "vocab_size": 30000
}

In [31]:
model = AutoModelForCausalLM.from_config(config)

The other thing we need to define is the `IPUTrainingArguments`, which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [32]:
training_args = IPUTrainingArguments(
    f"{model_checkpoint}-baby2",
    overwrite_output_dir=True,
    auto_loss_scaling=True,
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=micro_batch_size,
    per_device_eval_batch_size=micro_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=15,
    loss_scaling=16384,
    n_ipu=4,
    resume_from_checkpoint="./gpt2-baby/checkpoint-4500/",
    warmup_ratio=0.1,
    dataloader_drop_last=True,
    dataloader_num_workers=64,
    logging_steps=10,
    push_to_hub=False,
    save_total_limit=2
    # hub_model_id=f"username-or-organization/{model_checkpoint}-wikitext2",
)

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/gpt-finetuned-wikitext2"` or `"huggingface/gpt-finetuned-wikitext2"`).

Finally, we pass along all of those to the `IPUTrainer` class:

In [34]:
# model = AutoModelForCausalLM.from_pretrained("baby-gpt-2") # Alternatively, model can be continued to be pre-trained

In [35]:
from transformers import default_data_collator

trainer = IPUTrainer(
    model=model,
    ipu_config=ipu_config,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["eval"],
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    # resume_from_checkpoint = "./gpt2-baby/checkpoint-4500/"
)

Overriding IPU config: gradient_accumulation_steps=64,auto_loss_scaling=True
-------------------- Device Allocation --------------------
Token Embedding     --> IPU 0
Position Embedding  --> IPU 0
Layer 0             --> IPU 1
Layer 1             --> IPU 1
Layer 2             --> IPU 2
Layer 3             --> IPU 2
Layer 4             --> IPU 3
Layer 5             --> IPU 3
Head                --> IPU 0
-----------------------------------------------------------


And we can train our model:

In [36]:
print(transformers.__version__)

4.25.1


In [37]:
from transformers import AutoTokenizer,GPT2Tokenizer

In [45]:
model = AutoModelForCausalLM.from_pretrained("baby-gpt-2")
tokenizer=GPT2Tokenizer.from_pretrained("baby-gpt-2")
tokenizer

PreTrainedTokenizer(name_or_path='baby-gpt-2', vocab_size=30000, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'})

In [46]:
model.push_to_hub("gpt-2")
tokenizer.push_to_hub("gpt-2")

pytorch_model.bin:   0%|          | 0.00/272M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/iproskurina/gpt-2/commit/ce33a702d4056f8fe8c176443f092b58172c8285', commit_message='Upload tokenizer', commit_description='', oid='ce33a702d4056f8fe8c176443f092b58172c8285', pr_url=None, pr_revision=None, pr_num=None)

In [42]:
# tokenizer.save_pretrained("baby-gpt-2")

In [43]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 52, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
-------------------- Device Allocation --------------------
Token Embedding     --> IPU 0
Position Embedding  --> IPU 0
Layer 0             --> IPU 1
Layer 1             --> IPU 1
Layer 2             --> IPU 2
Layer 3             --> IPU 2
Layer 4             --> IPU 3
Layer 5             --> IPU 3
Head                --> IPU 0
-----------------------------------------------------------
Compiling Model...
Graph compilation: 100%|██████████| 100/100 [03:24<00:00]
Compiled/Loaded model in 229.32772605400532 secs
***** Ru

  0%|          | 0/11310 [00:00<?, ?it/s]

{'loss': 9.7867, 'learning_rate': 1.7683465959328031e-07, 'epoch': 0.01}
{'loss': 9.8477, 'learning_rate': 3.5366931918656063e-07, 'epoch': 0.03}
{'loss': 9.7594, 'learning_rate': 5.305039787798409e-07, 'epoch': 0.04}
{'loss': 9.6477, 'learning_rate': 7.073386383731213e-07, 'epoch': 0.05}
{'loss': 9.4656, 'learning_rate': 8.841732979664015e-07, 'epoch': 0.07}
{'loss': 9.343, 'learning_rate': 1.0610079575596817e-06, 'epoch': 0.08}
{'loss': 9.3031, 'learning_rate': 1.2378426171529622e-06, 'epoch': 0.09}
{'loss': 9.1258, 'learning_rate': 1.4146772767462425e-06, 'epoch': 0.11}
{'loss': 9.1359, 'learning_rate': 1.5915119363395226e-06, 'epoch': 0.12}
{'loss': 9.1211, 'learning_rate': 1.768346595932803e-06, 'epoch': 0.13}
{'loss': 9.0523, 'learning_rate': 1.945181255526083e-06, 'epoch': 0.15}
{'loss': 9.0109, 'learning_rate': 2.1220159151193635e-06, 'epoch': 0.16}
{'loss': 8.9391, 'learning_rate': 2.2988505747126437e-06, 'epoch': 0.17}
{'loss': 8.9156, 'learning_rate': 2.4756852343059245e-06,

Saving model checkpoint to gpt2-baby2/checkpoint-500


{'loss': 7.0914, 'learning_rate': 8.841732979664015e-06, 'epoch': 0.66}


...

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1432: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
-------------------- Device Allocation --------------------
Token Embedding     --> IPU 0
Position Embedding  --> IPU 0
Layer 0             --> IPU 1
Layer 1             --> IPU 1
Layer 2             --> IPU 2
Layer 3             --> IPU 2
Layer 4             --> IPU 3
Layer 5             --> IPU 3
Head                --> IPU 0
-----------------------------------------------------------
Configuration saved in gpt2-baby2/checkpoint-11000/ipu_config.json
Deleting older checkpoint [gpt2-baby2/checkpoint-10000] due to args.save_total_limit


{'loss': 4.2498, 'learning_rate': 5.894488653109343e-07, 'epoch': 14.6}
{'loss': 4.518, 'learning_rate': 5.698005698005699e-07, 'epoch': 14.62}
{'loss': 4.6605, 'learning_rate': 5.501522742902054e-07, 'epoch': 14.63}
{'loss': 4.6223, 'learning_rate': 5.305039787798409e-07, 'epoch': 14.64}
{'loss': 4.7449, 'learning_rate': 5.108556832694765e-07, 'epoch': 14.66}
{'loss': 4.7162, 'learning_rate': 4.912073877591119e-07, 'epoch': 14.67}
{'loss': 4.823, 'learning_rate': 4.7155909224874743e-07, 'epoch': 14.68}
{'loss': 4.9621, 'learning_rate': 4.51910796738383e-07, 'epoch': 14.69}
{'loss': 4.8561, 'learning_rate': 4.322625012280185e-07, 'epoch': 14.71}
{'loss': 4.4756, 'learning_rate': 4.12614205717654e-07, 'epoch': 14.72}
{'loss': 4.758, 'learning_rate': 3.9296591020728955e-07, 'epoch': 14.73}
{'loss': 4.5178, 'learning_rate': 3.733176146969251e-07, 'epoch': 14.75}
{'loss': 4.4219, 'learning_rate': 3.5366931918656063e-07, 'epoch': 14.76}
{'loss': 4.4131, 'learning_rate': 3.340210236761961e-0



Training completed. Do not forget to share your model on huggingface.co/models =)




{'loss': 4.4004, 'learning_rate': 0.0, 'epoch': 15.0}
{'train_runtime': 5038.1857, 'train_samples_per_second': 287.342, 'train_steps_per_second': 2.245, 'train_loss': 5.1905201771109635, 'epoch': 15.0}


TrainOutput(global_step=11310, training_loss=5.1905201771109635, metrics={'train_runtime': 5038.1857, 'train_samples_per_second': 287.342, 'train_steps_per_second': 2.245, 'train_loss': 5.1905201771109635, 'epoch': 15.0})

In [29]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 52, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
-------------------- Device Allocation --------------------
Token Embedding     --> IPU 0
Position Embedding  --> IPU 0
Layer 0             --> IPU 1
Layer 1             --> IPU 1
Layer 2             --> IPU 2
Layer 3             --> IPU 2
Layer 4             --> IPU 3
Layer 5             --> IPU 3
Head                --> IPU 0
-----------------------------------------------------------
Compiling Model...
Graph compilation: 100%|██████████| 100/100 [00:10<00:00]
Compiled/Loaded model in 19.17071347218007 secs
***** Run

  0%|          | 0/8140 [00:00<?, ?it/s]

{'loss': 9.9641, 'learning_rate': 2.457002457002457e-07, 'epoch': 0.01}
{'loss': 9.7258, 'learning_rate': 4.914004914004914e-07, 'epoch': 0.02}
{'loss': 9.7422, 'learning_rate': 7.371007371007371e-07, 'epoch': 0.04}
{'loss': 9.5406, 'learning_rate': 9.828009828009828e-07, 'epoch': 0.05}
{'loss': 9.4789, 'learning_rate': 1.2285012285012285e-06, 'epoch': 0.06}
{'loss': 9.2594, 'learning_rate': 1.4742014742014743e-06, 'epoch': 0.07}
{'loss': 9.2539, 'learning_rate': 1.7199017199017202e-06, 'epoch': 0.09}
{'loss': 9.3633, 'learning_rate': 1.9656019656019657e-06, 'epoch': 0.1}
{'loss': 9.1586, 'learning_rate': 2.2113022113022116e-06, 'epoch': 0.11}
{'loss': 9.0164, 'learning_rate': 2.457002457002457e-06, 'epoch': 0.12}
{'loss': 9.0344, 'learning_rate': 2.702702702702703e-06, 'epoch': 0.14}
{'loss': 8.9391, 'learning_rate': 2.9484029484029485e-06, 'epoch': 0.15}
{'loss': 8.875, 'learning_rate': 3.194103194103194e-06, 'epoch': 0.16}
{'loss': 8.6836, 'learning_rate': 3.4398034398034404e-06, 'e

Saving model checkpoint to gpt2-baby/checkpoint-500


{'loss': 6.9078, 'learning_rate': 1.2285012285012285e-05, 'epoch': 0.61}


...

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1432: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
-------------------- Device Allocation --------------------
Token Embedding     --> IPU 0
Position Embedding  --> IPU 0
Layer 0             --> IPU 1
Layer 1             --> IPU 1
Layer 2             --> IPU 2
Layer 3             --> IPU 2
Layer 4             --> IPU 3
Layer 5             --> IPU 3
Head                --> IPU 0
-----------------------------------------------------------
Configuration saved in gpt2-baby/checkpoint-8000/ipu_config.json
Deleting older checkpoint [gpt2-baby/checkpoint-7000] due to args.save_total_limit


{'loss': 4.9828, 'learning_rate': 3.549003549003549e-07, 'epoch': 9.84}
{'loss': 4.802, 'learning_rate': 3.2760032760032763e-07, 'epoch': 9.85}
{'loss': 5.0852, 'learning_rate': 3.003003003003003e-07, 'epoch': 9.86}
{'loss': 4.7496, 'learning_rate': 2.73000273000273e-07, 'epoch': 9.88}
{'loss': 4.5805, 'learning_rate': 2.457002457002457e-07, 'epoch': 9.89}
{'loss': 5.0316, 'learning_rate': 2.1840021840021843e-07, 'epoch': 9.9}
{'loss': 5.1301, 'learning_rate': 1.9110019110019112e-07, 'epoch': 9.91}
{'loss': 5.1613, 'learning_rate': 1.6380016380016382e-07, 'epoch': 9.93}
{'loss': 5.2223, 'learning_rate': 1.365001365001365e-07, 'epoch': 9.94}
{'loss': 5.1188, 'learning_rate': 1.0920010920010921e-07, 'epoch': 9.95}
{'loss': 4.6152, 'learning_rate': 8.190008190008191e-08, 'epoch': 9.96}
{'loss': 4.3162, 'learning_rate': 5.460005460005461e-08, 'epoch': 9.98}
{'loss': 4.7705, 'learning_rate': 2.7300027300027304e-08, 'epoch': 9.99}




Training completed. Do not forget to share your model on huggingface.co/models =)




{'loss': 4.6656, 'learning_rate': 0.0, 'epoch': 10.0}
{'train_runtime': 2876.2842, 'train_samples_per_second': 362.245, 'train_steps_per_second': 2.83, 'train_loss': 5.375703988789926, 'epoch': 10.0}


TrainOutput(global_step=8140, training_loss=5.375703988789926, metrics={'train_runtime': 2876.2842, 'train_samples_per_second': 362.245, 'train_steps_per_second': 2.83, 'train_loss': 5.375703988789926, 'epoch': 10.0})

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [ ]:
# import math
# eval_results = dtrainer.evaluate()
# print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

The perplexity is still quite high since for this demo we trained on a small dataset for a small number of epochs. For a real LM training, you  would need a larger dataset and more epochs.

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
# trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sgugger/my-awesome-model")
```